In [331]:
!pip install git+https://github.com/m-a-huber/AuToMATo

  Cloning https://github.com/m-a-huber/AuToMATo to /tmp/pip-req-build-v8k_49kc
  Running command git clone --filter=blob:none --quiet https://github.com/m-a-huber/AuToMATo /tmp/pip-req-build-v8k_49kc
  Resolved https://github.com/m-a-huber/AuToMATo to commit 84dbc424e50792448288f1fe408c736530f83a04
ERROR: git+https://github.com/m-a-huber/AuToMATo does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [264]:
import os

import pickle
import time
from multiprocessing import Pool
from time import sleep

import networkx as nx
import numpy as np
from sklearn.cluster import HDBSCAN, KMeans
from tqdm.notebook import trange, tqdm
from scripts import centroids_graph_builder, graph_osm_loader, utils
import gudhi
from automato import Automato
from gudhi.clustering.tomato import Tomato

import kmapper as km
from kmapper.jupyter import display

Примеры как резолвить комьюнити

In [265]:
#resolve_communities
def resolve_communities(H: nx.Graph, **params) -> list[set[int]]:
    communities = nx.community.louvain_communities(H,
                                                   seed=1534,
                                                   weight='length',
                                                   resolution=params['r'])
    return utils.validate_cms(H, communities)

In [266]:
def resolve_by_hdbscan(H: nx.Graph):
    def f(a,b):
        u = int(a[2])
        v = int(b[2])
        if (u,v) in H.edges() or (v,u) in H.edges():
            return H.edges()[(u,v)]['length']
        return float('inf')
    scan = HDBSCAN(metric=f, min_samples=1, max_cluster_size=30,n_jobs = 10)
    x = np.array([[d['x'], d['y'], u] for u, d in g.nodes(data=True)])
    y = scan.fit_predict(x)
    communities = {}
    for i, u in enumerate(g.nodes):
        cls = y[i]
        if cls not in communities:
            communities[cls] = set()
        communities[cls].add(u)
    communities = [communities[cls] for cls in communities]
    del scan
    return utils.validate_cms(H, communities)

In [267]:
# def resolve_by_mapper(H: nx.Graph):
#     id2node = {i:u for i, u in enumerate(H.nodes())}
#     mapper = km.KeplerMapper(verbose=0)
#     x = np.array([[d['x'], d['y']] for u, d in H.nodes(data=True)])
#     proj_data = mapper.fit_transform(x)
#     g = mapper.map(proj_data, x, clusterer= HDBSCAN())
#     cms = []
#     scan = HDBSCAN(metric=f, min_samples=1, max_cluster_size=30,n_jobs = 10)
#     x = np.array([[d['x'], d['y'], u] for u, d in g.nodes(data=True)])
#     
#     all_nodes = set()
#     for i, (k,v) in enumerate(dict(g['nodes']).items()):
#         # print(k)
#         for n in v:
#             all_nodes.add(n)
#         cms.append(set([id2node[id_node] for id_node in v ]))
#     # mapper.visualize(g, path_html="make_circles_keplermapper_output.html",
#     #              title="make_circles(n_samples=5000, noise=0.03, factor=0.3)")
#     print(len(all_nodes), len(x))
#     return utils.validate_cms(H, cms)


In [268]:
def resolve_by_part(g):
    cms = nx.community.kernighan_lin_bisection(g, weight='length')
    cms = utils.validate_cms(g, cms)
    res = []
    for c in cms:
        if len(c) < 100:
            res.append(c)
        else:
            gg = g.subgraph(c)
            rr = resolve_by_part(gg)
            rr = utils.validate_cms(gg, rr)
            res.extend(rr)
    return utils.validate_cms(g, res)

In [269]:
def k_clique_communities(g, k =2):
    cms= nx.community.k_clique_communities(g,k )
    res = utils.validate_cms(g, cms)
    assert nx.community.is_partition(g, res)
    return res

In [270]:
def girvan_newman(g):
    comp = nx.community.girvan_newman(g)
    for i, communities in tqdm(enumerate(comp), total= 800):
        cms = communities
        if len(cms)>800:
            break
    return utils.validate_cms(g, cms)

In [271]:
def resolve_by_kmeans(H: nx.Graph):
    scan = KMeans(n_clusters=100)
    x = np.array([[d['x'], d['y'], u] for u, d in g.nodes(data=True)])
    y = scan.fit_predict(x)
    communities = {}
    for i, u in enumerate(g.nodes):
        cls = y[i]
        if cls not in communities:
            communities[cls] = set()
        communities[cls].add(u)
    communities = [communities[cls] for cls in communities]
    del scan
    return utils.validate_cms(H, communities)

In [326]:
def tomato_resolver(H, r = 0.1):
    xx = {(d['x'], d['y']):u for u, d in H.nodes(data=True)}
    
    def f(a,b):
        
        u = a[2]
        v = b[2]
        
        if (u,v) in H.edges() or (v,u) in H.edges():
            return H.edges()[(u,v)]['length']
        return float('inf')
    x = np.array([[d['x'], d['y'], u] for u, d in H.nodes(data=True)])
        
    ex1 = Tomato(
        # metric = f,
            input_type="points",
        # n_jobs = 10,
        # p = 1,
            graph_type="radius",
            density_type="KDE",
            # n_clusters=800,
            r=r,
        )
    
    ex1.fit(x)
    cms = {}
    ll = list(ex1.labels_)
    for i, u in enumerate(g.nodes()):
        l = ll[i]
        if l not in cms:
            cms[l] = set()
        cms[l].add(u)
    cms_1 = []
    for c in cms.values():
        cms_1.append(c)
    return utils.validate_cms(g, cms_1)

In [273]:
# find path between two points
def find_path_length_h(
        g0: nx.Graph,
        g1: nx.Graph,
        cms: list[set[int]] | tuple[set[int]],
        from_node: int,
        to_node: int) -> float:
    from_cluster = g0.nodes()[from_node]['cluster']
    to_cluster = g0.nodes()[to_node]['cluster']
    
    def h(a, b):
        da = g0.nodes()[cls2c[a]]
        db = g0.nodes()[cls2c[b]]
        return ((da['x'] - db['x']) ** 2 + (da['y'] - db['y']) ** 2) ** 0.5 / 360 * 2 * np.pi * 6371.01 * 1000 * 0.56
    
    path = nx.astar_path(g1, from_cluster, to_cluster,heuristic=h, weight='length')
    cls = set([u for u in path])
    g = centroids_graph_builder.extract_cluster_list_subgraph(g0, cls, cms)
    return nx.single_source_dijkstra(g, from_node, to_node, weight='length')

In [274]:
GRAPH_ID = 'R2555133' # R13470549 R2555133

In [275]:
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID) # загрузка графа
len(g.nodes), len(g.edges)

(17883, 26972)

In [288]:
utils.get_opt_cluster_count(len(g.nodes)) # оптимальное количество кластеров

858

In [329]:
cms = tomato_resolver(g, r =0.8)#(g, r = 26.5)
print(len(cms))

17883


In [304]:
cls2n =  centroids_graph_builder.get_cls2n(g) # мапа кластер к соседним кластерам 

In [324]:
g1, cls2c = centroids_graph_builder.build_center_graph(g, cms, cls2n, log=True)

find edges:   0%|          | 13/11159 [00:02<30:11,  6.15it/s]


KeyboardInterrupt: 

In [306]:
path = utils.get_path('pouits', f'points_{GRAPH_ID}.pickle')

if os.path.exists(path):
    with open(path, 'rb') as fp:
        points = pickle.load(fp)
        fp.close()
else:
    points = [utils.get_node_for_initial_graph_v2(g) for _ in trange(1000, desc='generate points')]
    with open(path, 'wb') as fp:
        pickle.dump(points, fp)
        fp.close()

In [307]:
NUM_ITERATION = 2 # чтобы уменьшить ошибку при вычислении времени выполнения, при каждом замере время меряется для NUM_ITERATION повторений
WORKER = 4 # количество потоков


def do_calc(data):
    pps, i = data

    stat = {}
    stat['l'] = []
    stat['h_l'] = []
    stat['p'] = []
    stat['h_p'] = []
    stat['time_l'] = []
    stat['time_h'] = []

    stat['delta'] = []
    # чисто чтобы tqdm нормально прогрузился 
    sleep(i/10)
    print('start', i)
    
    for p1, p2 in tqdm(pps, desc='find paths', position=i):
        if (p1, p2) in stat:
            continue
        # класический дейкстра
        l, p = None, None
        start = time.time()
        for i in range(NUM_ITERATION):
            l, p = nx.single_source_dijkstra(g, p1, p2, weight='length')
        time_l = time.time() - start
        
        # иерархический
        h_l, h_p = None, None
        start = time.time()
        for _ in range(NUM_ITERATION):
            h_l, h_p = find_path_length_h(g, g1,cms, p1, p2)
        time_h = time.time() - start
        
        delta = (h_l - l) / l * 100
        stat['l'].append(l) # длина обычного пути
        stat['h_l'].append(h_l) # длина иерархического пути
        stat['p'].append(p) # обычный путь
        stat['h_p'].append(h_p) # иерархический путь
        stat['delta'].append(delta) # разница в длине
        stat['time_l'].append(time_l) # обычное время 
        stat['time_h'].append(time_h) # иерархическое
    return stat

data = [([p for p in points[i::WORKER]], i) for i in range(WORKER)]
with Pool(WORKER) as p:
    res = p.map(do_calc, data)

stat = {}
for l in res:
    for d in l:
        if d not in stat:
            stat[d] = []
        stat[d].extend(l[d])
print('err_mean:', np.mean(stat['delta']))
print('err_min:', np.min(stat['delta']))
print('err_max:', np.max(stat['delta']))
print(np.mean(np.array(stat['time_l']) / np.array(stat['time_h'])))

start 0


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

start 1


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

start 2


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

start 3


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

err_mean: 7.158752195805673
err_min: 0.0
err_max: 115.87059109940981
2.986050325043698
